### <span style="color:rgb(139,69,19)">CIMPA School Research School "Control, Optimization, and Model Reduction in Machine Learning"</span>

### <span style="color:rgb(139,69,19)">Optimization for Machine Learning - C. W. Royer</span>


# <span style="color:rgb(139,69,19)">Lab 01 - Around gradient descent</span>

#### <span style="color:rgb(139,69,19)">Preliminary remarks</span>

- This notebook and the subsequent ones used in this course mix Python code and text/LaTeX blocks. They can be run offline on any computer where Python and Jupyter are installed, or online using Google Colab (requires a Google account).

- All code blocks from this notebook are meant to be run in the order that they are given. In particular, the first block below must be run first in order to import the necessary toolboxes.

- All the notebooks from this course rely on Python and the NumPy library. A basic yet very useful tutorial on NumPy is freely available
[here](https://sebastianraschka.com/pdf/books/dlb/appendix_f_numpy-intro.pdf).

In [ ]:
# Preamble: useful toolboxes, librairies, functions, etc.

# Display
%matplotlib inline
import matplotlib.pyplot as plt

from math import sqrt # Square root
from math import ceil # Ceil integer operator
from math import log # Logarithm function

# NumPy - Matrix and vector structures
import numpy as np # NumPy library
from numpy.random import multivariate_normal, randn, uniform # Probability distributions

# SciPy - Efficient mathematical calculation
from scipy.linalg import toeplitz # Toeplitz matrices
from scipy.linalg import norm # Euclidean norm
from scipy.linalg import svdvals # Singular value decomposition
from scipy.linalg import qr # QR decomposition from linear algebra
from scipy.optimize import check_grad # Numerical check of derivatives
from scipy.optimize import fmin_l_bfgs_b # An efficient minimization routine in moderate dimensions

# <span style="color:rgb(139,69,19)">Introduction</span>

In this session, we illustrate the behavior of gradient descent techniques on linear regression, arguably the most classical data analysis task. The nice structure of such problems (smoothness, convexity) allows to showcase the behavior of gradient descent.

# <span style="color:rgb(139,69,19)">1 - Data and problem class</span>

We consider a dataset $\{(\mathbf{x}_i,y_i)\}_{i=1,\dots,n}$, where $\mathbf{x}_i \in \mathbb{R}^d$ and $y_i \in \mathbb{R}$, available in the form of:

- a feature matrix $\mathbf{X} \in \mathbb{R}^{n \times d}$;
- and a vector of labels $\mathbf{y} \in \mathbb{R}^n$. 

Given this dataset, we will seek a linear model parameterized by a vector $\mathbf{w}$ that explains the data according to a certain loss function $\ell$. This results in the following formulation:
$$
    \min_{\mathbf{w} \in \mathbb{R}^d} f(\mathbf{w}) = \frac{1}{n} \sum_{i=1}^n f_i(\mathbf{w}), \qquad f_i(\mathbf{w}) = \ell(\mathbf{x}_i^T\mathbf{w},y_i) + \frac{\lambda}{2}\|\mathbf{w}\|^2_2,
$$
where $\lambda \ge 0$ is a regularization parameter associated with an $\ell_2$ regularization term. In the first part of the notebook, we will use $\lambda=0$, but provide the generic framework for subsequent sections.

The dataset will be produced according to the procedure below.

In [ ]:
# Data generation.
# This code is inspired by a generator proposed by A. Gramfort from INRIA.

def simu_linmodel(wtrue, n, std=1., corr=0.5):
    """
    Simulation values obtained by a linear model with additive noise
    
    Parameters
    ----------
    wtrue : np.ndarray, shape=(d,)
        The true coefficients of the model
    
    n : int
        Sample size
    
    std : float, default=1.
        Standard-deviation of the noise

    corr : float, default=0.5
        Correlation of the feature matrix
    """    
    d = wtrue.shape[0]
    cov = toeplitz(corr ** np.arange(0, d))
    X = multivariate_normal(np.zeros(d), cov, size=n)
    noise = std * randn(n)
    y = X.dot(wtrue) + noise
    return X, y

The data is thus produced from a linear model corrupted with noise: $\mathbf{y} = \mathbf{X} \mathbf{w}^* + \mathbf{\epsilon}$, where $\mathbf{\epsilon}$ follows a Gaussian distribution. Our goal will thus be to learn a linear model from the data.

## <span style="color:rgb(139,69,19)">1.1 Linear regression</span>

In *linear regression*, we seek a linear model that explains the data based on minimizing a least-squares objective:
$$
    \mathrm{minimize}_{\mathbf{w} \in \mathbb{R}^d} f(\mathbf{w}) 
    := \frac{1}{2 n} \|\mathbf{X} \mathbf{w} - \mathbf{y}\|^2_2 + \frac{\lambda}{2}\|\mathbf{w}\|^2_2
$$ 
for some $\lambda \ge 0$.

### <span style="color:rgb(139,69,19)">Observations</span> 

- The function $f$ is quadratic.
- The function $f$ is $L$-smooth with $L = \frac{\|\mathbf{X}^T \mathbf{X}\|_2}{n}+\lambda$. 
- The function $f$ is convex, and $(\sigma_{\min}(\mathbf{X})+\lambda)$-strongly convex when $\sigma_{\min}(\mathbf{X})+\lambda>0$, where $\sigma_{\min}(\mathbf{X})$ denotes the minimum singular value of $\mathbf{X}$.

### <span style="color:rgb(139,69,19)">Mathematical details</span> 

One can rewrite the objective function as
$$
    f(\mathbf{w}) = \frac{1}{2}\mathbf{w}^T \left(\frac{\mathbf{X}^T \mathbf{X}}{n} + \lambda \mathbf{I}_d\right)\mathbf{w} - \frac{1}{n}\mathbf{y}^T \mathbf{X} \mathbf{w} + \frac{1}{2} \mathbf{y}^T \mathbf{y}.
$$
The gradient of the objective function above is given by:
$$
    \nabla f(\mathbf{w}) = \left( \frac{\mathbf{X}^T \mathbf{X}}{n} + \lambda \mathbf{I}_d \right)\mathbf{w} - \frac{1}{n}\mathbf{X}^T \mathbf{y}
    = \frac{1}{n}\mathbf{X}^T (\mathbf{X} \mathbf{w} - \mathbf{y}) + \lambda \mathbf{w}.
$$
For any $\mathbf{w},\mathbf{v} \in \mathbb{R}^d$, the formula for the gradient gives:
$$
    \|\nabla f(\mathbf{w}) -\nabla f(\mathbf{v})\| 
    = \left\|\frac{\mathbf{X}^T \mathbf{X}}{n} \mathbf{w} + \lambda \mathbf{w} - \frac{\mathbf{X}^T \mathbf{X}}{n}\mathbf{v} - \lambda \mathbf{v} \right\|_2 
    \le \frac{\|\mathbf{X}^T \mathbf{X}\|_2}{n}\|\mathbf{w}-\mathbf{v}\|_2 + \lambda \|\mathbf{w}-\mathbf{v}\|_2.
$$
The gradient $\nabla f$ is thus $\left(\frac{\|\mathbf{X}^T \mathbf{X}\|_2}{n}+\lambda\right)$-Lipschitz continuous.

## <span style="color:rgb(139,69,19)">1.2 Logistic regression</span>

In *logistic regression*, we still consider a linear model, but for classification purposes (here $y_i \in \{-1,1\}$). We use a loss function better suited for classification:
$$
    \mathrm{minimize}_{\mathbf{w} \in \mathbb{R}^d} f(\mathbf{w}) 
    := \frac{1}{n} \sum_{i=1}^n f_i(\mathbf{w}), \qquad 
    f_i(\mathbf{w})=\log(1+\exp(-y_i \mathbf{x}_i^T \mathbf{w}))+\frac{\lambda}{2}\|\mathbf{w}\|^2_2,
$$
where $\lambda \ge 0$ is a regularization parameter.

### <span style="color:rgb(139,69,19)">Observations</span> 

- For any $\mathbf{w} \in \mathbb{R}^d$, one has
$$
\nabla f(\mathbf{w}) = \frac{1}{n}\sum_{i=1}^n  -\frac{y_i}{1 + \exp(y_i \mathbf{x}_i^T \mathbf{w})} \mathbf{x}_i + \lambda \mathbf{w}.
$$
- The function $f$ is convex and even $\lambda$-strongly convex when $\lambda>0$.
- The function $f$ is $L$-smooth with $L =\frac{\|\mathbf{X}^T \mathbf{X}\|_2}{4n}+\lambda$.

### <span style="color:rgb(139,69,19)">Details</span> 

1. The derivative of $t \mapsto \log(1+\exp(-t))$ is $t \mapsto \frac{-\exp(-t)}{1+\exp(-t)} = -\frac{1}{1+\exp(t)}$. Combining this with the linear function $\mathbf{w} \mapsto y_i \mathbf{x}_i^T \mathbf{w}$, we get for every $i$ that
$$
\nabla f_i(\mathbf{w}) = - \frac{y_i}{1 + \exp(y_i \mathbf{x}_i^T \mathbf{w})} \mathbf{x}_i + \lambda \mathbf{w}.
$$
and 
$$
\nabla f(\mathbf{w}) = \frac{1}{n}\sum_{i=1}^n  -\frac{y_i}{1 + \exp(y_i \mathbf{x}_i^T \mathbf{w})} \mathbf{x}_i + \lambda \mathbf{w}
$$
2. One way to obtain a Lipschitz constant for the gradient is to bound the second-order derivative, which is given by:
$$
\nabla^2 f(\mathbf{w}) = \frac{1}{n}\sum_{i=1}^n  
\frac{\exp(-y_i \mathbf{x}_i^T \mathbf{w})}{(1 + \exp(-y_i \mathbf{x}_i^T \mathbf{w}))^2} \mathbf{x}_i \mathbf{x}_i^T + \lambda I,
$$
where $I$ is the identity matrix. The result follows by noticing that $t \mapsto \tfrac{e^{t}}{(1+\exp(t))^2}$  is always less than or equal to $\tfrac{1}{4}$ (its value at the origin), we obtain that
$$
\| \nabla^2 f(\mathbf{w}) \| \le L:=\frac{\|\mathbf{X}^T \mathbf{X}\|_2}{4n}+\lambda,
$$
hence $\nabla f$ is $L$-Lipschitz continuous.

## <span style="color:rgb(139,69,19)">1.3 Python class for regression problems</span>

The Python class below encodes the objective and the gradient of both regression problems, using the formulas from the previous sections. Note that formulas for the Lipschitz constants as well as a strong convexity parameter are provided.

In [ ]:
# Python class for regression problems
class RegPb(object):
    '''
        A class for regression problems with linear models.
        
        Attributes:
            X: Data matrix (features)
            y: Data vector (labels)
            n,d: Dimensions of X
            loss: Loss function to be considered in the regression
                'l2': Least-squares loss
                'logit': Logistic loss
            lbda: Regularization parameter
    '''
   
    # Instantiate the class
    def __init__(self, X, y,lbda=0,loss='l2'):
        self.X = X
        self.y = y
        self.n, self.d = X.shape
        self.loss = loss
        self.lbda = lbda
        
    
    # Objective value
    def fun(self, w):
        if self.loss=='l2':
            return norm(self.X.dot(w) - self.y) ** 2 / (2. * self.n) + self.lbda * norm(w) ** 2 / 2.
        elif self.loss=='logit':
            yXw = self.y * self.X.dot(w)
            return np.mean(np.log(1. + np.exp(-yXw))) + self.lbda * norm(w) ** 2 / 2.

    
    # Full gradient computation
    def grad(self, w):
        if self.loss=='l2':
            return self.X.T.dot(self.X.dot(w) - self.y) / self.n + self.lbda * w
        elif self.loss=='logit':
            yXw = self.y * self.X.dot(w)
            aux = 1. / (1. + np.exp(yXw))
            return - (self.X.T).dot(self.y * aux) / self.n + self.lbda * w
    

    # Lipschitz constant for the gradient
    def lipgrad(self):
        if self.loss=='l2':
            L = norm(self.X, ord=2) ** 2 / self.n + self.lbda
        elif self.loss=='logit':
            L = norm(self.X, ord=2) ** 2 / (4. * self.n) + self.lbda
        return L
    
    # ''Strong'' convexity constant (could be zero if self.lbda=0)
    def cvxval(self):
        if self.loss=='l2':
            s = svdvals(self.X)
            mu = min(s)**2 / self.n # More efficient than computing ||A^T A||
            return mu + self.lbda
        elif self.loss=='logit':
            return self.lbda

The first script below generates two problem instances (one for each class), the second checks the implementation of the derivatives, and the third one computes an approximate solution for each problem.

In [ ]:
# Generate the problem instances - we use moderate sizes but those will serve our purpose

d = 50
n = 1000
idx = np.arange(d)
lbda = 1. / n ** (0.5)

# Fix random seed for reproducibility
np.random.seed(1)

# Ground truth coefficients of the model, chosen so that the magnitude of the entries decays
w_model_truth = (-1)**idx * np.exp(-idx / 10.)

Xlin, ylin = simu_linmodel(w_model_truth, n, std=1., corr=0.1)
Xlog, ylog = simu_linmodel(w_model_truth, n, std=1., corr=0.7)
ylog = np.sign(ylog) # Taking the logarithm for binary classification

pblinreg = RegPb(Xlin, ylin,lbda,loss='l2')
pblogreg = RegPb(Xlog, ylog,lbda,loss='logit')

## <span style="color:rgb(139,69,19)">1.4 Numerical estimates of $\min$ and $\mathrm{argmin}$</span>

In this lab, we work with relatively simple loss functions (and a moderate number of data points): we can thus efficiently compute a solution using a second-order method. This provides us with a target objective value as well as a target vector of weights.

In [ ]:
# Use L-BFGS-B to determine a solution for both problems

w_init = np.zeros(d)
# Compute the optimal solution for linear regression
w_min_lin, f_min_lin, _ = fmin_l_bfgs_b(pblinreg.fun, w_init, pblinreg.grad, args=(), pgtol=1e-30, factr =1e-30)
print("Linear regression:")
print("\t Numerical minimal value:",f_min_lin)
print("\t Numerical minimum gradient norm:",norm(pblinreg.grad(w_min_lin)))

# Compute the optimal solution for logistic regression
w_min_log, f_min_log, _ = fmin_l_bfgs_b(pblogreg.fun, w_init, pblogreg.grad, args=(), pgtol=1e-30, factr =1e-30)
print("Logistic regression:")
print("\t Numerical minimal value:",f_min_log)
print("\t Numerical minimum gradient norm:",norm(pblogreg.grad(w_min_log)))

These solutions will enable us to study the behavior of the distance to optimality in terms of function values 
$f(\mathbf{w}_k)-f^*$ and iterates $\|\mathbf{w}_k -\mathbf{w}^*\|$. 

*Note: Since $\|\nabla f(\mathbf{w}_k)\|^2 \ge 2 \mu (f(\mathbf{w}_k)-f^*)$ for a $\mu$-strongly convex, continuously differentiable function with optimal value $f^*$, the gradient could also be used as an upper estimate of the distance to optimality in terms of function values.*

# <span style="color:rgb(139,69,19)">2 - Gradient descent implementation</span>

We will investigate several techniques for selecting the stepsize in a predetermined or adaptive fashion in gradient descent. Those are by no means exhaustive, but give a nice overview of existing options (see upcoming lectures for more details about the theory behind those methods). We will consider the three following options:

- *Constant stepsize:* $\alpha_k = \frac{\bar{\alpha}}{L}$, where $L$ is the Lipschitz constant for $\nabla f$ (assumed to be known in that case).

- *Decreasing stepsize:* $\alpha_k = \frac{\bar{\alpha}}{(k+1)^a}$, where $a>0$ is given as input

- *Armijo-type line search:* $\alpha_k=\frac{\bar{\alpha}}{2^{j_k}}$, where $j_k$ is the smallest nonnegative integer such that
$$
f\left(\mathbf{w}_k-\frac{\bar{\alpha}}{2^{j_k}}\nabla f(\mathbf{w}_k)\right) < f(\mathbf{w}_k) - 0.0001 \frac{\alpha}{2^{j_k}}\|\nabla f(\mathbf{w}_k)\|^2_2.
$$
In practice, the line search should stop if the decrease condition is satisfied or drops below a certain value, e.g. $\frac{\bar{\alpha}}{2^{j_k}} < 10^{-10}$. Note that the parameters $2$,$0.0001$ and $10^{-10}$ are set for simplicity.


Those are implemented in the template algorithm below.

In [ ]:
# Gradient descent
def gd_algo(w0,problem,wopt,stepchoice=0,stepbar=1, n_iter=1000, verbose=False): 
    """
        An implementation of gradient descent with several stepsize rules.
        
        Inputs:
            w0: Initial point
            problem: Problem structure
                problem.fun(w) Objective value
                problem.grad(w) Gradient
                problem.lipgrad() Lipschitz constant for the gradient
            wopt: Target point for the optimization (approximate optimum computed beforehand)
            stepchoice: Rule for choosing the stepsize (see above)
                0: Constant equal to 1/L where L is a Lipschitz constant for the gradient
                a>0: Decreasing, set to 1/((k+1)**a)
                -1: Armijo line search
            stepbar: Initial step size (used when stepchoice = 1)
            n_iter: Maximum iteration number
            verbose: Boolean value indicating whether iteration-level information should be displayed.
      
        Outputs:
            w_output: Last iterate of the method
            objvals: History of function values (Numpy array of length n_iter)
            distits: History of distances to the target point (Numpy array of length n_iter)
            ngvals: History of gradient norms (Numpy array of length n_iter)
            stepsize_variant: String describing the stepsize variant
            
    """
    
    ############
    # Initialization

    # History of function values
    objvals = []
    
    # History of gradient norms
    ngvals = []
    
    # History of distances to the target
    distits = []
    
    # Lipschitz constant
    L = problem.lipgrad()
    
    # Initial value of the incumbent, a.k.a. current iterate
    w = w0.copy()

    # Initialize the iteration count
    k=0    
    
    # Initial function value
    obj = problem.fun(w) 
    objvals.append(obj);
    
    # Initial gradient
    g = problem.grad(w)
    ng = norm(g)
    ngvals.append(ng)
    
    # Distance between the current point and the optimal point
    dist = norm(w-wopt)
    distits.append(dist)
    
    # Label for stepsize choice
    if stepchoice==0:
        stepsize_variant="Cst "+f"{stepbar/L:.2f}"
    elif stepchoice>0:
        stepsize_variant="Dec "+f"{stepbar:.2f}"+"/(k+1)^"+f"{stepchoice:.2f}"
    else:
        stepsize_variant="Lis (Init "+f"{stepbar:.2f}"+")"

    # Plot initial values 
    if verbose:
        print("Gradient descent:")
        print(' | '.join([name.center(8) for name in ["iter", "fval", "dist","stepsize"]]))
    
    ####################
    # Main loop
    while (k < n_iter):
        
        #####################################################################################
        
        # 1 - Define the stepsize s based on k (iteration index), L (Lipschitz constant), step0 (initial value)
        # and g (the function)
        if stepchoice==0:
            # Constant stepsize
            s = (stepbar/L)
        elif stepchoice>0:
            # Decreasing stepsize
            s = stepbar/((k+1)**stepchoice)
        elif stepchoice==-1:
            # Line search (inner while loop)
            s = stepbar
            while (problem.fun(w-s*g) >= obj - 0.0001*s*(ng**2)) and (s>1e-10):
                s = 0.5*s
        
        # 2 - Perform the gradient descent iteration using the stepsize s and the gradient g
        w[:] = w - s*g
      
        
        ####################################################################################
        
        
        # Plot relevant information
        if verbose:        
            print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8),("%.2e" % dist).rjust(8),("%.2e" % s).rjust(8)]))
        
        # Compute values associated with the next iterate
        obj = problem.fun(w)
        objvals.append(obj)
        g = problem.grad(w)
        ng = norm(g)
        ngvals.append(ng)
        dist = norm(w-wopt)
        distits.append(dist)
        
        # Increase iteration counter
        k += 1
    
    # End main loop
    ######################
    
    # Outputs
    w_output = w.copy()
    return w_output, np.array(objvals), np.array(distits), np.array(ngvals),stepsize_variant

### <span style="color:rgb(139,69,11)">Hands-on! Experiments on linear regression</span>

**a) Run gradient descent when using the four following rules on linear regression:**

- $\alpha_k = \frac{1}{L}$;

- $\alpha_k = \frac{1}{k+1}$;

- $\alpha_k = \frac{1}{\sqrt{k+1}}$;

- $\alpha_k$ **chosen through Armijo line search with $\bar{\alpha}=1$.**

**b)Compare the convergence curves in terms of function values $\{f(x_k)\}$ and $\{f(x_k)-f^*_{lin}\}$, where 
$f^*_{lin}$ is the value computed in Section 1.**

**c) Plot the gradient norms and the distances to optimum as a function of the number of iterations. Are those plots consistent with the others? Do they bring additional insights about the methods?**

In [ ]:
# Part a)

# Run four variants of gradient descent using the same initial point
w0 = np.zeros(d)
_, obj_a, dist_a, ngrad_a, sv_a = gd_algo(w0,pblinreg,w_min_lin,stepchoice=0,stepbar=1, n_iter=50)
_, obj_b, dist_b, ngrad_b, sv_b = gd_algo(w0,pblinreg,w_min_lin,stepchoice=1,stepbar=1, n_iter=50)
_, obj_c, dist_c, ngrad_c, sv_c = gd_algo(w0,pblinreg,w_min_lin,stepchoice=0.5,stepbar=1, n_iter=50)
_, obj_d, dist_d, ngrad_d, sv_d = gd_algo(w0,pblinreg,w_min_lin,stepchoice=-1,stepbar=1, n_iter=50)

# Best/Final objective value
print("Final objective value for GD -"+sv_a+" \t",obj_a[-1])
print("Final objective value for GD -"+sv_b+" \t",obj_b[-1])
print("Final objective value for GD -"+sv_c+" \t",obj_c[-1])
print("Final objective value for GD -"+sv_d+" \t",obj_d[-1])


In [ ]:
# Part b)

# We begin by plotting the change in the function value (in log scale) as a function of the iteration number
plt.figure(figsize=(7, 5))
plt.semilogy(obj_a, label="GD "+sv_a, lw=2)
plt.semilogy(obj_b, label="GD "+sv_b, lw=2)
plt.semilogy(obj_c, label="GD "+sv_c, lw=2)
plt.semilogy(obj_d, label="GD "+sv_d, lw=2)
plt.title("Objective behavior", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Objective function (log)", fontsize=14)
plt.legend()

# We can also plot the change in function value relatively to the target value 
# Note: In log scale, the values f(w)-vmin close to 0 do not appear


#### Plot the behavior of all four methods in terms of relative objective value
plt.figure(figsize=(7, 5))
plt.semilogy(obj_a-f_min_lin, label="GD "+sv_a, lw=2)
plt.semilogy(obj_b-f_min_lin, label="GD "+sv_b, lw=2)
plt.semilogy(obj_c-f_min_lin, label="GD "+sv_c, lw=2)
plt.semilogy(obj_d-f_min_lin, label="GD "+sv_d, lw=2)
plt.title("Objective-Optimum behavior", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Objective function-Target (log)", fontsize=14)
plt.legend()

In [ ]:
# Part c)

#### Plot the behavior of all four methods in terms of gradient norm
plt.figure(figsize=(7, 5))
plt.semilogy(ngrad_a, label="GD "+sv_a, lw=2)
plt.semilogy(ngrad_b , label="GD "+sv_b, lw=2)
plt.semilogy(ngrad_c, label="GD "+sv_c, lw=2)
plt.semilogy(ngrad_d, label="GD "+sv_d, lw=2)
plt.title("Gradient norm behavior", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Gradient norm (log)", fontsize=14)
plt.legend()

#### Plot the behavior of all four methods in terms of distance to numerical optimum
plt.figure(figsize=(7, 5))
plt.semilogy(dist_a, label="GD "+sv_a, lw=2)
plt.semilogy(dist_b , label="GD "+sv_b, lw=2)
plt.semilogy(dist_c, label="GD "+sv_c, lw=2)
plt.semilogy(dist_d, label="GD "+sv_d, lw=2)
plt.title("Distance to numerical minimum", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Distance (log)", fontsize=14)
plt.legend()

### <span style="color:rgb(139,69,19)">Hands-on! Experiments on logistic regression</span>

**a) Run the code below to try out a few values for the stepsize.**

**b) Uncomment the last lines and try to find the best possible rule for the stepsize on logistic regression.**

In [ ]:
###############################################
# Experiments on logistic regression
###############################################

# Plotting the behavior of all methods
# Run four variants of gradient descent using the same initial point
# For the last variant, try to improve over the previous four
w0 = np.zeros(d)
_, obj_a, dist_a, ngrad_a, sv_a = gd_algo(w0,pblogreg,w_min_log,stepchoice=?,stepbar=?, n_iter=50)
_, obj_b, dist_b, ngrad_b, sv_b = gd_algo(w0,pblogreg,w_min_log,stepchoice=?,stepbar=?, n_iter=50)
_, obj_c, dist_c, ngrad_c, sv_c = gd_algo(w0,pblogreg,w_min_log,stepchoice=?,stepbar=?, n_iter=50)
_, obj_d, dist_d, ngrad_d, sv_d = gd_algo(w0,pblogreg,w_min_log,stepchoice=?,stepbar=?, n_iter=50)
_, obj_e, dist_e, ngrad_e, sv_e = gd_algo(w0,pblogreg,w_min_log,stepchoice=-1,stepbar=10, n_iter=50)


#### Plot the behavior of all four methods in terms of relative objective value
plt.figure(figsize=(7, 5))
plt.semilogy(obj_a-f_min_log, label="GD "+sv_a, lw=2)
plt.semilogy(obj_b-f_min_log, label="GD "+sv_b, lw=2)
plt.semilogy(obj_c-f_min_log, label="GD "+sv_c, lw=2)
plt.semilogy(obj_d-f_min_log, label="GD "+sv_d, lw=2)
plt.semilogy(obj_e-f_min_log, label="GD "+sv_e, lw=2)
plt.title("Objective-Optimum behavior", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Objective function-Target (log)", fontsize=14)
plt.legend()

# <span style="color:rgb(139,69,19)">3 - Regularization</span>

In this section, we focus on linear regression problems, and consider the use of several regularization terms.

## <span style="color:rgb(139,69,19)">3.1 $\ell_1$ regularization</span>

The LASSO estimator is defined as the solution of
$$
    \mathrm{minimize}_{\mathbf{w} \in \mathbb{R}^d}\ f(\mathbf{w}) + \lambda \|\mathbf{w}\|_1,
    \quad \mathrm{with} \quad
    f(\mathbf{w}):= \frac{1}{2 n} \|\mathbf{X} \mathbf{w} - \mathbf{y}\|^2_2
$$ 
Here $\mathbf{X} \in \mathbb{R}^{n \times d}$ and $\mathbf{y} \in \mathbb{R}^n$ define the data-fitting part of the data, and $\lambda \ge 0$.

In [ ]:
# Data generation from a sparse ground truth 

# Reproducibility
np.random.seed(1)

# Ground truth with only 5% nonzero entries (built from the previously used ground truth)
d = 200
n = 200
s = round(0.95*min(d,n))
idx = np.arange(d)
wtrue = (-1)**idx * np.exp(-idx / 10.)
ip = np.random.permutation(d)
wtrue[ip[0:s]]=0


# Columns of X follow a normal distribution N(0,1/n)

X = multivariate_normal(np.zeros(d), (1/n)*np.identity(d), size=n)
Xw = X.dot(wtrue)
std = (1/n)*(norm(Xw)**2)
noise = std * randn(n)
y = Xw + noise

# Instantiate the (smooth) part of the problem
pbsparse = RegPb(X,y)

## <span style="color:rgb(139,69,19)">3.2 Proximal gradient/ISTA</span>

We recall the basics of the *Iterative Soft-Thresholding Algorithm* (ISTA) below.

Given an iterate $\mathbf{x}_k$ and a positive stepsize $\alpha_k$, the gradient step with respect to the smooth 
part of the objective is given by:
$$
    \mathbf{g}_k = \mathbf{w}_k - \alpha_k \nabla f(\mathbf{w}_k),
$$
where we recall that $f$ denotes the smooth part of 
$f_{\ell_1}(\mathbf{w}) = f(\mathbf{w}) + \lambda \|\mathbf{w}\|_1$.

An iteration of proximal gradient applies the proximal operator 
$\mathbf{w}_{k+1}=\mbox{prox}_{\alpha_k\lambda\|\cdot\|_1}(\mathbf{g}_k)$, which is equivalent to
$$
    \mathbf{w}_{k+1} \in \mathrm{argmin}_{\mathbf{w} \in \mathbb{R}^d} \left\{ f(\mathbf{w}_k) + \nabla f(\mathbf{w}_k)^\top (\mathbf{w}-\mathbf{w}_k) + \frac{1}{2\alpha_k}\|\mathbf{w}-\mathbf{w}_k\|^2 + \lambda \|\mathbf{w}\|_1 \right\}. 
$$ 
 
The proximal subproblem solution has a close form, that can be stated componentwise as:
$$
    \forall i=1,\dots,d, \quad [\mathbf{w}_{k+1}]_i \; = \; 
    \left\{
        \begin{array}{ll}
            [\mathbf{g}_k]_i + \lambda \alpha_k &\mathrm{if\ } [\mathbf{g}_k]_i < -\lambda \alpha_k \\
            [\mathbf{g}_k]_i - \lambda \alpha_k &\mathrm{if\ } [\mathbf{g}_k]_i > \lambda \alpha_k \\
            0 &\mathrm{otherwise.}
        \end{array}
    \right.
$$

In [ ]:
# Implementation of ISTA
def ista(w0,problem,lbda,stepchoice=0,step0=1, n_iter=1000,verbose=False): 
    """
        A code for ISTA with various step choices.
        
        Inputs:
            w0: Initial vector
            problem: Problem structure
                problem.fun(x) evaluates the objective function (which is assumed to be a finite sum of functions) at a given vector w
                problem.grad(x) evaluates the gradient of the smooth part of the objective function at a vector w
                problem.lipgrad() returns the Lipschitz constant for the gradient
                problem.cvxval() returns the strong convexity constant
            lbda: Regularization parameter
            stepchoice: Strategy for computing the stepsize 
                0: Constant step size equal to step0/L
                1: Step size decreasing in step0/(k+1)
                2: Step size decreasing in step0/sqrt(k+1)
            step0: Initial steplength (only used when stepchoice is not 0)
            n_iter: Number of iterations
            
        Outputs:
            w_output: Final iterate of the method
            objvals: History of function values (output as a Numpy array of length n_iter at most)
    """   
    
    
    ############
    # Initial step: Compute and plot some initial quantities

    # objective history
    objvals = []
    
    # Lipschitz constant
    L = problem.lipgrad()
    
    # Initial value of current iterate   
    w = w0.copy()

    # Initialize iteration counter
    k=0    
    
    # Current objective
    obj = problem.fun(w) 
    objvals.append(obj);

    step=step0 # Initialize constant stepsize
    threshold=0 # Initialize threshold for proximal step
    
    # Display initial quantities
    if verbose:
        print("ISTA:")
        print(' | '.join([name.center(8) for name in ["iter", "fval"]]))
        print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8)]))
    
    
    ##########################
    # Main loop
    while (k < n_iter):
        # Compute the gradient of the smooth part
        g = problem.grad(w)
        # Select the stepsize
        if stepchoice==0:
            step = 1/L
        elif (stepchoice==1):
            step = step0/(k+1)
        else:
            step = step0/(sqrt(k+1))
        
        # Compute the proximal gradient step
        for i in range(problem.d):
            vali = w[i]-step*g[i]
            threshold = step*lbda
            if vali < -threshold:
                w[i] = vali+threshold
            elif vali > threshold:
                w[i] = vali-threshold
            else:
                w[i] = 0
            
        # Update objective value and iteration index
        obj = problem.fun(w)
        objvals.append(obj);
        k += 1
        if verbose:
            print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8)]))       
        
    # Output 
    w_output = w.copy()
          
    return w_output, np.array(objvals)

### <span style="color:rgb(139,69,19)">Hands-on! ISTA</span>

**Try out the proximal gradient method with several values of $\lambda$ in $[0,1]$. Can you 
find a good value for this parameter?**

In [ ]:
# Testing l1 regularization
#
# ADD VALUES FOR lbda HERE!
lvals = []
nlbda = len(lvals)
w0 = np.ones(d)
Wsol = np.zeros((d,nlbda))
dist_truth = np.zeros(nlbda)

plt.figure(figsize=(7, 5))
for i in range(nlbda):
    lbda =lvals[i]
    Wsol[:,i], obj_is = ista(w0,pbsparse,lbda,stepchoice=0,step0=1, n_iter=100)
    dist_truth[i] = norm(Wsol[:,i]-wtrue,1)
    print(f'Nonzero coefficients with lbda={lbda:.2e}'+f': {np.count_nonzero(Wsol[:,i]):d}')
    print(f'Difference with truth, lbda={lbda:.2e}'+f': {dist_truth[i]:.2e}')
    plt.semilogy(obj_is, label=f'lbda={lbda:2e}', lw=2)
plt.title("Performance of ISTA", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Objective (log scale)", fontsize=14)
plt.legend(loc=1)


plt.figure(figsize=(7, 5))
for i in ip[s:]:
    plt.semilogx(lvals,Wsol[i,:],label="w_"+str(i),c=np.random.rand(3,), lw=2)
plt.title("ISTA solutions", fontsize=16)
plt.xlabel("Regularization", fontsize=14)
plt.ylabel("Magnitude", fontsize=14)
plt.legend(ncol=4,loc=1)

plt.figure(figsize=(7,5))
plt.semilogx(lvals,dist_truth/norm(wtrue,1),lw=2)
plt.title("Error to ground truth",fontsize=16)
plt.xlabel("Regularization", fontsize=14)
plt.ylabel("Magnitude",fontsize=14)

# <span style="color:rgb(139,69,19)">4 - Robust linear regression and subgradient method</span>

In this final section, we look at a linear regression problem based on a nonsmooth loss.

Given a dataset $(\mathbf{X},\mathbf{y})$ for regression, we again aim at computing a linear model parameterized by a vector $\mathbf{w} \in \mathbb{R}^d$. In the previous sections, we considered the least-squares loss
$$
    \mathrm{minimize}_{\mathbf{w} \in \mathbb{R}^d} 
    \frac{1}{2n}\left\| \mathbf{X}\mathbf{w}-\mathbf{y} \right\|_2^2 
    =\frac{1}{2n}\sum_{i=1}^n (\mathbf{x}_i^T \mathbf{w}-y_i)^2.
$$

When the dataset contains outliers, linear regression can produce poor results. An alternate optimization formulation, less sensitive to outliers, consists in replacing the $\ell_2$ norm by the $\ell_1$ norm in the objective, yielding
$$
    \mathrm{minimize}_{\mathbf{w} \in \mathbb{R}^d} 
    \frac{1}{n}\left\| \mathbf{X}\mathbf{w}-\mathbf{y} \right\|_1 
    = \frac{1}{n}\sum_{i=1}^n |\mathbf{x}_i^T \mathbf{w}-y_i|.
$$

## <span style="color:rgb(139,69,19)">4.1 Dataset</span>

The code below implements a dataset with outliers, which can be observed directly in dimension $1$.

In [ ]:
# Data generation
# This generating technique is adapted from J. Duchi (Stanford)
def data_outliers(d, n, rng):
    
    """
    Linear model with a proportion of outliers
    
    Inputs
    ----------
        d : Parameters
    
        n : Samples
        
        rng : Random number generator
        
    Outputs
    ----------
    
        X,y: Feature matrix and label vector
    """    
    
    X = rng.multivariate_normal(np.zeros(d), np.eye(d), size=n)
    u = rng.multivariate_normal(np.zeros(d), np.eye(d))
    noise = rng.normal(0.0,1.0,size=n)
    y = X.dot(u) + noise*np.power(np.abs(noise),3)
    return X, y

In [ ]:
# Visualization of 1D data
rng= np.random.default_rng(4)

x0,y0=data_outliers(1,200,rng)
plt.figure(figsize=(7, 5))
plt.scatter(x0,y0, label="Samples", lw=2)
plt.title("Dataset with outliers", fontsize=16)
plt.legend()

In the next section, we will use a higher-dimensional (yet still small-dimensional) dataset.

In [ ]:
# Dataset to be used in the next section

d=50
n=100

rng= np.random.default_rng(4)

X,y=data_outliers(d,n,rng)

For any $\mathbf{w} \in \mathbb{R}^d$, a subgradient of $f$ is given by
$$
    g(\mathbf{w}) = \frac{1}{n}\mathbf{X}^T \mbox{sign}(\mathbf{X}\mathbf{w}-\mathbf{y}), 
    \qquad 
    \mbox{sign}(\mathbf{X}\mathbf{w}-\mathbf{y})
    =
    \begin{bmatrix} 
    \mbox{sign}(\mathbf{x}_1^T \mathbf{w}-y_1) \\ 
    \vdots \\
    \mbox{sign}(\mathbf{x}_n^T \mathbf{w}-y_n)
    \end{bmatrix},
    \qquad
    \mbox{sign}(t) = 
        \left\{
        \begin{array}{ll}
            1 &\mbox{if } t>0 \\
            -1 &\mbox{if } t<0 \\
            0 &\mbox{if } t=0.
        \end{array}
        \right.
$$

In [ ]:
# Tailored subgradient algorithm for robust linear regression
def sub_grad(X,y,w0,stepchoice=0,step0=1, n_iter=4000,verbose=False): 
    """
        Subgradient algorithm for robust linear regression with l1 loss.
        
        Inputs:
            X: Data matrix (features)
            y: Data vector (labels)
            w0: Initial point
            stepchoice: Stepsize choice
                0: Constant proportional to 1/L (L Lipschitz constant for the gradient)
                a>0: Decreasing, set to 1/((k+1)**a)
            step0: Initial stepsize
            n_iter: Maximum number of iterations
            verbose: Plot information at the iteration level?
            
        Outputs:
            w_output: Last iterate
            objvals: History of function values at the iterates
            objavg: History of function values at the averatges
    """
    
    ############
    # Initialization
    objvals = []
    objavg = []
    
    
    # Dimensions
    (n,d) = X.shape
    
    # Initial point
    w = w0.copy()
    
    # Average of iterates
    wavg=w0.copy()

    k=0
    
    obj = norm(X.dot(w)-y,1) 
    objvals.append(obj)
    objavg.append(obj)
    
    if verbose:
        # Optional display
        print("Subgradient, robust regression",n)
        print(' | '.join([name.center(8) for name in ["iter", "fval", "favg"]]))
        print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8),("%.2e" % obj).rjust(8)]))
    
    ################
    # Main loop
    while (k < n_iter):
        
        
        # Compute subgradient
        sg = (1/n)*X.T@np.sign(X.dot(w)-y)
            
        if stepchoice==0:
            w[:] = w - step0 * sg
        elif stepchoice>0:
            sk = float(step0/((k+1)**stepchoice))
            w[:] = w - sk * sg
            
        obj = norm(X.dot(w)-y,1)
        
        # Maintain a running average of the iterates
        wavg = k/(k+1) *wavg + w/(k+1) 
        obj_wavg = norm(X.dot(wavg)-y,1)
        

        k += 1
        
        # Record objective values for both the iterate sequence and the average sequence
        objvals.append(obj)
        objavg.append(obj_wavg)
        if verbose:
            print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8),("%.2e" % obj_wavg).rjust(8)]))       
    
    # End main loop
    #################
    
    # Solution
    w_output = w.copy()
    
    return w_output, np.array(objvals), np.array(objavg)

### <span style="color:rgb(139,69,19)">Hands on! Subgradient</span>

**Run the subgradient method below with different constant stepsize choices. Use at least one value <1 and one value $\ge$ 1.**

In [ ]:
# Experiments on subgradient methods

w0 = np.zeros(d)

# Choose stepsizes greater and smaller than 1
cst_step = []


nvals = len(cst_step)
nits = 4000

objs = np.zeros((nits+1,nvals))
avgs = np.zeros((nits+1,nvals))

for val in range(nvals):
    _, objs[:,val], avgs[:,val] = sub_grad(X,y,w0,stepchoice=0,step0=cst_step[val],n_iter=nits)

In [ ]:
# Plotting objective values (iterates+average)
plt.figure(figsize=(7, 5))
plt.set_cmap("RdPu")
for val in range(nvals):
    plt.semilogy(objs[:,val], label="alpha="+str(cst_step[val]), lw=2)
plt.title("Objective values (iterates)", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Objective (log)", fontsize=14)
plt.legend(loc=1)

plt.figure(figsize=(7, 5))
plt.set_cmap("RdPu")
for val in range(nvals):
    plt.semilogy(avgs[:,val], label="alpha="+str(cst_step[val]), lw=2)
plt.title("Objective values (average)", fontsize=16)
plt.xlabel("#Iterations", fontsize=14)
plt.ylabel("Objective (log)", fontsize=14)
plt.legend(loc=1)

In [ ]:
# Version 1.1 - C. W. Royer, February 2025.